# Intro to SQL
Learn SQL for working with databases, using Google BigQuery to scale to massive datasets.

## Getting Started With SQL and BigQuery

In [1]:
from google.cloud import bigquery

## Select, From & Where

In [ ]:
# Create a "Client" object.


## Group by, Having & Count  

GROUP BY tells SQL how to apply aggregate functions (like COUNT()), it doesn't make sense to use GROUP BY without an aggregate function. Similarly, if you have any GROUP BY clause, then all variables must be passed to either a

- GROUP BY command, or
- an aggregation function.

## Order By

## As & With

## Joining Data

## 